In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def haar_lwt_1d_decompose(data, level):
    """
    Performs a 1D Haar Lifting Wavelet Transform decomposition for a given number of levels.

    The lifting scheme for Haar wavelet involves:
    1. Split: Separate the signal into even and odd indexed samples.
    2. Predict: Calculate detail coefficients (high-frequency) by subtracting
       the even samples from the odd samples.
    3. Update: Calculate approximation coefficients (low-frequency) by adding
       half of the detail coefficients to the even samples.

    Args:
        data (np.ndarray): The 1D input data array.
        level (int): The number of decomposition levels to perform.

    Returns:
        tuple: A tuple containing:
            - final_approximation_coeffs (np.ndarray): The approximation coefficients
              at the highest decomposition level. This represents the "reduced values".
            - detail_coeffs_info_list (list): A list of tuples, where each tuple contains:
                (detail_coeffs (np.ndarray), original_length_at_this_level (int))
              The list is ordered from the lowest decomposition level (finest details)
              to the highest decomposition level (coarsest details).
              The `original_length_at_this_level` is crucial for accurate reconstruction
              when dealing with signals that had odd lengths at certain stages.
    """
    # Ensure data is a float numpy array for calculations
    current_coeffs = np.array(data, dtype=float)
    
    # List to store detail coefficients and their original lengths at each level
    detail_coeffs_info_list = [] 

    # Perform decomposition for the specified number of levels
    for i in range(level):
        # Store the original length of the signal at the current level before any padding
        original_len_at_this_level = len(current_coeffs)

        # Pad the current coefficients if their length is odd.
        # This ensures that 'even' and 'odd' parts have compatible lengths.
        if original_len_at_this_level % 2 != 0:
            # Pad with a single zero at the end
            padded_coeffs = np.pad(current_coeffs, (0, 1), 'constant')
        else:
            padded_coeffs = current_coeffs

        # Step 1: Split - Separate into even and odd indexed samples
        even = padded_coeffs[::2]
        odd = padded_coeffs[1::2]

        # Step 2: Predict - Calculate detail coefficients (d_j = odd - even)
        detail = odd - even

        # Step 3: Update - Calculate approximation coefficients (s_j = even + d_j / 2)
        approximation = even + detail / 2

        # Store the detail coefficients along with the original length of the signal
        # at this level (before padding), which is needed for accurate reconstruction.
        detail_coeffs_info_list.append((detail, original_len_at_this_level))
        
        # The approximation coefficients become the input for the next decomposition level
        current_coeffs = approximation

        # Stop decomposition if the approximation coefficients become too small (e.g., single element),
        # as further decomposition is not meaningful.
        if len(current_coeffs) < 2:
            print(f"Warning: Stopped decomposition at level {i+1} because signal length became too small.")
            break

    # The final `current_coeffs` are the approximation coefficients at the highest level
    return current_coeffs, detail_coeffs_info_list

def haar_lwt_1d_reconstruct(final_approximation_coeffs, detail_coeffs_info_list):
    """
    Reconstructs a 1D signal from its Haar Lifting Wavelet Transform coefficients.

    The inverse lifting scheme for Haar wavelet involves:
    1. Inverse Update: Revert the update step (even = s_j - d_j / 2).
    2. Inverse Predict: Revert the predict step (odd = d_j + even).
    3. Merge: Combine the reconstructed even and odd parts to form the signal
       at the previous level, trimming any padding if necessary.

    Args:
        final_approximation_coeffs (np.ndarray): The approximation coefficients
                                                at the highest decomposition level.
        detail_coeffs_info_list (list): A list of tuples, where each tuple contains:
                                        (detail_coeffs (np.ndarray), original_length_at_this_level (int))
                                        This list should be in the order from lowest
                                        to highest level of detail (as returned by decompose).

    Returns:
        np.ndarray: The reconstructed 1D signal, which should ideally be identical
                    to the original input data.
    """
    # Start reconstruction from the highest level approximation coefficients
    reconstructed_coeffs = np.array(final_approximation_coeffs, dtype=float)

    # Reconstruct level by level, processing detail coefficients from highest to lowest level
    # (i.e., in reverse order of how they were generated during decomposition)
    for detail_info in reversed(detail_coeffs_info_list):
        detail, original_len_at_this_level = detail_info

        # Step 1: Inverse Update
        # Revert the approximation calculation to get the 'even' part back
        even = reconstructed_coeffs - detail / 2

        # Step 2: Inverse Predict
        # Revert the detail calculation to get the 'odd' part back
        odd = detail + even

        # Step 3: Merge - Combine the even and odd parts
        # The combined length will be the length of the signal at the previous level,
        # potentially including padding if the original signal at that level was odd.
        combined_padded_len = len(even) + len(odd)
        temp_reconstructed = np.empty(combined_padded_len, dtype=float)
        
        # Place even samples at even indices and odd samples at odd indices
        temp_reconstructed[::2] = even
        temp_reconstructed[1::2] = odd
        
        # Trim the reconstructed signal to its original length at this level.
        # This removes any padding that was added during decomposition.
        reconstructed_coeffs = temp_reconstructed[:original_len_at_this_level]

    return reconstructed_coeffs

def save_coefficients_to_files(output_dir, final_approximation, detail_coeffs_info_list):
    """
    Saves all Haar LWT coefficients (approximation and all detail levels) into a single .csv file,
    with each coefficient type in its own column.

    Args:
        output_dir (str): The directory where the coefficient file will be saved.
                          If the directory does not exist, it will be created.
        final_approximation (np.ndarray): The final approximation coefficients
                                          (representing the reduced values).
        detail_coeffs_info_list (list): A list of tuples (detail_coeffs, original_length)
                                        for each decomposition level.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Prepare a list of all coefficient arrays to be combined into columns
    all_coeff_arrays = [final_approximation]
    headers = ['Approximation']

    for i, (detail, _) in enumerate(detail_coeffs_info_list):
        all_coeff_arrays.append(detail)
        headers.append(f'Detail_Level_{i+1}')

    # Find the maximum length among all coefficient arrays for consistent column size
    max_len = 0
    if all_coeff_arrays: # Ensure there's at least one array to check length
        max_len = max(len(arr) for arr in all_coeff_arrays)

    # Pad shorter arrays with NaN to match the maximum length
    # This ensures all columns have the same number of rows
    padded_coeff_arrays = []
    for arr in all_coeff_arrays:
        if len(arr) < max_len:
            padded_arr = np.pad(arr, (0, max_len - len(arr)), 'constant', constant_values=np.nan)
        else:
            padded_arr = arr
        padded_coeff_arrays.append(padded_arr)

    # Stack the padded arrays horizontally to form a 2D array (columns)
    if padded_coeff_arrays:
        combined_coeffs_2d = np.column_stack(padded_coeff_arrays)
    else:
        combined_coeffs_2d = np.array([[]]) # Handle case where no coefficients are generated

    # Create the header string for the CSV file
    header_str = ','.join(headers)

    # Save the combined coefficients to a single CSV file
    combined_filepath = os.path.join(output_dir, 'all_haar_lwt_coeffs_columns.csv')
    np.savetxt(combined_filepath, combined_coeffs_2d, delimiter=',', header=header_str, comments='')
    print(f"Saved all Haar LWT coefficients (in columns) to: {combined_filepath}")

# --- Example Usage ---
if __name__ == "__main__":
    # 1. Define sample 1D data
    # Example with even length
    data_even = np.array([10, 20, 30, 40, 50, 60, 70, 80])
    # Example with odd length
    data_odd = np.array([1, 2, 3, 4, 5, 6, 7])

    df = pd.read_csv('Data_August_Renewable.csv')
    data = df['Speed'].values
    
    # Choose which data to use for demonstration
    # input_data = data_odd 
    input_data = data
    n_levels = 2 # Number of decomposition levels

    print(f"Original 1D Data: {input_data}")
    print(f"Decomposition Levels: {n_levels}")
    print("-" * 30)

    # 2. Perform N-level Haar LWT decomposition
    final_approx, detail_coeffs_info = haar_lwt_1d_decompose(input_data, n_levels)

    print("\n--- Decomposition Results ---")
    print(f"Final Approximation Coefficients (Reduced Values): {final_approx}")
    print("Detail Coefficients per Level (from finest to coarsest detail):")
    for i, (detail, original_len) in enumerate(detail_coeffs_info):
        print(f"  Level {i+1} Detail: {detail} (Original length at this level: {original_len})")
    print("-" * 30)

    # 3. Save coefficients to files
    output_directory = 'haar_lwt_coeffs'
    save_coefficients_to_files(output_directory, final_approx, detail_coeffs_info)
    print("-" * 30)

    # 4. Perform reconstruction
    reconstructed_data = haar_lwt_1d_reconstruct(final_approx, detail_coeffs_info)

    print("\n--- Reconstruction Results ---")
    print(f"Reconstructed Data: {reconstructed_data}")
    
    # 5. Verify reconstruction accuracy
    # Use np.isclose for floating-point comparisons
    is_reconstruction_accurate = np.allclose(input_data, reconstructed_data)
    print(f"Is Reconstruction Accurate? {is_reconstruction_accurate}")
    if not is_reconstruction_accurate:
        print(f"Difference: {input_data - reconstructed_data}")
    print("-" * 30)

    # --- How to extend to N-dimensional data (Conceptual) ---
    print("\n--- Extending to N-Dimensional Data ---")
    print("For N-dimensional data (e.g., 2D images), the 1D transform is typically applied")
    print("iteratively along each dimension. For example, for a 2D image:")
    print("1. Apply 1D LWT to each row.")
    print("2. Apply 1D LWT to each column of the resulting coefficients.")
    print("This process can be generalized for 3D or higher dimensions.")
    print("You would need to reshape your N-dimensional data into 1D segments (rows, columns, slices),")
    print("apply this 1D function, and then reshape back. The reconstruction would follow the inverse steps.")
    print("-" * 30)


Original 1D Data: [7.343 8.013 8.293 ... 9.517 9.46  9.309]
Decomposition Levels: 2
------------------------------

--- Decomposition Results ---
Final Approximation Coefficients (Reduced Values): [8.09975 9.22075 9.9     ... 8.84425 9.15425 9.42825]
Detail Coefficients per Level (from finest to coarsest detail):
  Level 1 Detail: [ 0.67   0.457  0.235 ...  0.106  0.09  -0.151] (Original length at this level: 4176)
  Level 2 Detail: [ 0.8435  0.0845  0.647  ...  0.2315  0.1375 -0.0875] (Original length at this level: 2088)
------------------------------
Saved all Haar LWT coefficients (in columns) to: haar_lwt_coeffs/all_haar_lwt_coeffs_columns.csv
------------------------------

--- Reconstruction Results ---
Reconstructed Data: [7.343 8.013 8.293 ... 9.517 9.46  9.309]
Is Reconstruction Accurate? True
------------------------------

--- Extending to N-Dimensional Data ---
For N-dimensional data (e.g., 2D images), the 1D transform is typically applied
iteratively along each dimension.